In [1]:
import pandas as pd
import numpy
import os
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

df = pd.read_csv('./data/Speed+Dating+Data.csv',low_memory=False,encoding='cp1252')
display(df.head())
display(df.shape)
display(df.columns)
display(df[['age','gender','match']].describe())

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


(8378, 195)

Index(['iid', 'id', 'gender', 'idg', 'condtn', 'wave', 'round', 'position',
       'positin1', 'order',
       ...
       'attr3_3', 'sinc3_3', 'intel3_3', 'fun3_3', 'amb3_3', 'attr5_3',
       'sinc5_3', 'intel5_3', 'fun5_3', 'amb5_3'],
      dtype='object', length=195)

,age,gender,match
count,8283.000000,8378.000000,8378.000000
mean,26.358928,0.500597,0.164717
std,3.566763,0.500029,0.370947
min,18.000000,0.000000,0.000000
25%,24.000000,0.000000,0.000000
50%,26.000000,1.000000,0.000000
75%,28.000000,1.000000,0.000000
max,55.000000,1.000000,1.000000


# Remarques sur les données et data cleaning

1. Il y a une différence de notation entre les groupes sur les batchs 6 à 9 (notation de 1 à 10 au lieu de 100 points à distribuer)
2. Il y a un manque important de données principalement sur le remplissage des formulaires notamment après le speed dating 


Pour que les données restent cohérentes dans l'analyse qui va suivre je vais filtrer les données pour exclure les wave 6 à 9


In [2]:
print(f'Nombre de valeurs vides : \n{df.isna().sum().sort_values(ascending=False).head(10)}')
print(f"{len(df.columns[df.isna().sum() > 1000])} colonnes ont plus de 1000 valeurs manquantes")
print(f"{len(df.columns[df.isna().sum() > 2000])} colonnes ont plus de 2000 valeurs manquantes")
print(f"{len(df.columns[df.isna().sum() > 3000])} colonnes ont plus de 3000 valeurs manquantes")
print(f"{len(df.columns[df.isna().sum() > 4000])} colonnes ont plus de 4000 valeurs manquantes")

list_to_delete = df.columns[df.isna().sum() > 4000].to_list()

filterd_df = df[~df['wave'].isin([6,7,8,9])]
filterd_df = filterd_df.drop(list_to_delete,axis=1)
print(f"shape après le filtrage : {filterd_df.shape}")


Nombre de valeurs vides : 
num_in_3    7710
numdat_3    6882
expnum      6578
sinc7_2     6423
amb7_2      6423
shar7_2     6404
attr7_2     6394
fun7_2      6394
intel7_2    6394
fun5_3      6362
dtype: int64
94 colonnes ont plus de 1000 valeurs manquantes
83 colonnes ont plus de 2000 valeurs manquantes
71 colonnes ont plus de 3000 valeurs manquantes
65 colonnes ont plus de 4000 valeurs manquantes
shape après le filtrage : (6816, 130)



### Variables clés (extrait du Data Key)

- **match** (1/0) : match réciproque (deux "Yes").  
- **dec** / **dec_o** : décision du participant / du partenaire ce soir-là (1=Yes, 0=No).  
- **attr, sinc, intel, fun, amb, shar** : notes attribuées au partenaire lors du *date* (1–10).  
- **attr_o, ...** : notes que **le partenaire** vous attribue.  
- **pf_o_att** : préférences déclarées du partenaire (vagues/échelles différentes selon *wave*).  
- **samerace** : 1 si même origine ethnique, 0 sinon.  
- **age, age_o** : âge du participant / du partenaire.  
- **gender** : 0 = Femme, 1 = Homme.  
- **order** : numéro du rendez-vous dans la soirée.  
- **attr3_1, ..., amb3_1** : **auto-évaluations** (Time 1).  
- **attr1_1, ..., shar1_1** : **importance déclarée** des attributs (Time 1).


On se concentre sur TIME 1 car variabilité trop importante après



In [25]:
print("Distribution de gender")
fig = px.bar(filterd_df['gender'].value_counts().reset_index().replace(1,'Hommes').replace(0,'Femmes'),x='gender',y='count',title='Distribution gender')
fig.show()

Distribution de gender


In [26]:
distrib_metier = px.bar(filterd_df['career'].value_counts()[:40].reset_index(),x='career',y='count',title='Distribution career (40 premiers)')
distrib_metier.show()

# Distribution de l'étnie des participants

In [5]:
race = filterd_df['race'].value_counts().reset_index().sort_values(by='count',ascending=False)
print(race)
# affectation manuel de la concordance pour plus de clareté. attention si les données changent il faut changer ici
race['str_race'] = ['Caucasian','Asian/Pacific','Latino','Other','African American']
print(race)
distrib_wave = px.bar(race,x='str_race',y='count', title='Distribution sur etnies')
distrib_wave.show()

   race  count
0   2.0   3786
1   4.0   1649
2   3.0    569
3   6.0    446
4   1.0    308
   race  count          str_race
0   2.0   3786         Caucasian
1   4.0   1649     Asian/Pacific
2   3.0    569            Latino
3   6.0    446             Other
4   1.0    308  African American


### Analyse des niveaux de revenus

In [6]:
incomes = df['income'].str.removesuffix('.00').str.replace(',','')
incomes = incomes.dropna()
print(f"La moyenne des revenus estimés est : {round(incomes.astype(float).mean(),2)} $")
print(f"La médiane des revenus estimés est : {round(incomes.astype(float).median(),2)} $")
print(f"Max income : {incomes.astype(float).max()}")
print(f"Min income : {incomes.astype(float).min()}")


La moyenne des revenus estimés est : 44887.61 $
La médiane des revenus estimés est : 43185.0 $
Max income : 109031.0
Min income : 8607.0


## Qui dit oui pour un second RDV le plus souvent ?

In [7]:
male = filterd_df[filterd_df['gender']==1]
female = filterd_df[filterd_df['gender']==0]

print(f"Les hommes ont dit {round((male['dec'] == 1).sum()/male['dec'].count()*100,2)} % oui à un second RDV")
print(f"Les femmes ont dit {round((female['dec'] == 1).sum()/female['dec'].count()*100,2)} % oui à un second RDV")
print(f"% de match : {round((df['match'] == 1).sum()/df['match'].count()*100,2)} %")

Les hommes ont dit 45.53 % oui à un second RDV
Les femmes ont dit 37.88 % oui à un second RDV
% de match : 16.47 %


## Quel est l'attribut le moins désirable

In [8]:
female_attributes = female[['attr1_1','sinc1_1','intel1_1','fun1_1','amb1_1','shar1_1']]
moyenne_femme = female_attributes.mean().sort_values(ascending=True)

male_attributes = male[['attr1_1','sinc1_1','intel1_1','fun1_1','amb1_1','shar1_1']]
moyenne_male = male_attributes.mean().sort_values(ascending=True)

attr_men = pd.DataFrame(moyenne_male,columns=['moyenne']).reset_index()
attr_women = pd.DataFrame(moyenne_femme,columns=['moyenne']).reset_index()

fig_men = px.bar(attr_men,x='index',y="moyenne")
fig_women = px.bar(attr_women,x='index',y="moyenne")

fig_attr = make_subplots(rows=1,cols=2,subplot_titles=('hommes','femmes'))
fig_attr.add_trace(fig_men.data[0],row=1,col=1)
fig_attr.add_trace(fig_women.data[0],row=1,col=2)
fig_attr.update_layout(title_text="Moyenne des notes par genre")
fig_attr.show()

Pour les femmes et les hommes la qualité la moins désirable est l'ambition

L'Attirance physique est en 1ère position chez les hommes alors qu'en 2ème chez les femmes.

Qu'en est il de la réalité ? 

On cherche donc à trouver la moyenne des notes quand la ou le participant à dit oui

In [9]:
note_attribue_female_to_male = female[['attr','sinc','intel','fun','amb','shar','dec']]
note_attribue_male_to_female = male[['attr','sinc','intel','fun','amb','shar','dec']]

reel_impact_femmes = note_attribue_female_to_male[note_attribue_female_to_male['dec'] == 1].mean().sort_values(ascending=False)
reel_impact_femmes = pd.DataFrame(reel_impact_femmes.reset_index(name=('moyenne')))
fig_women_impact = px.bar(reel_impact_femmes,x='index',y="moyenne")

reel_impact_hommes = note_attribue_male_to_female[note_attribue_male_to_female['dec'] == 1].mean().sort_values(ascending=False)
reel_impact_hommes = pd.DataFrame(reel_impact_hommes.reset_index(name=('moyenne')))
fig_men_impact = px.bar(reel_impact_hommes,x='index',y="moyenne")

fig_impact = make_subplots(rows=1,cols=2,subplot_titles=('hommes','femmes'))
fig_impact.add_trace(fig_men_impact.data[0],row=1,col=1)
fig_impact.add_trace(fig_women_impact.data[0],row=1,col=2)
fig_impact.update_layout(title_text="Moyenne des notes par genre et ayant dit oui à un second rdv")
fig_impact.show()


L'attirance arrive en 5eme position quand les femmes ont dit oui à un seccond rdv

L'attirance arrive en 3eme position quand les hommes ont dit oui à un seccond rdv

### Y a t'il une corrélation entre le match et si les deux participants sont de la même etnie ?

In [11]:
print(filterd_df[['shar','samerace','dec']].corr()['dec'])
corr_etnie = filterd_df[['shar','samerace','dec']].corr()
corr_df_fig = px.imshow(corr_etnie, text_auto=True,
    color_continuous_scale="RdBu_r",
    title="Matrice de corrélation (Pearson)",
    zmin=-1, zmax=1)
corr_df_fig.show()
print("\nTaux de second rdv par même origine (0 : non - 1 : oui):")
print(filterd_df.groupby('samerace')['dec'].mean())

filterd_df['shar_supp_5'] = (filterd_df['shar'] >= 6).astype(int)

print("\nTaux de second rdv par intérêts partagés (>=6) (0 : non - 1 : oui) :")
print(filterd_df.groupby('shar_supp_5')['dec'].mean())


shar        0.410090
samerace    0.025921
dec         1.000000
Name: dec, dtype: float64



Taux de second rdv par même origine (0 : non - 1 : oui):
samerace
0    0.406882
1    0.433083
Name: dec, dtype: float64

Taux de second rdv par intérêts partagés (>=6) (0 : non - 1 : oui) :
shar_supp_5
0    0.272491
1    0.604377
Name: dec, dtype: float64


## L'origine n'a pas l'air d'avoir plus d'importance que le fait de partager les mêmes intérets

# Est ce que les gens percoivent correctement leur propre valeur ?

In [22]:
evaluation = ['attr3_1', 'sinc3_1', 'intel3_1', 'fun3_1', 'amb3_1',"attr_o", "sinc_o", "intel_o", "fun_o", "amb_o"]
auto_eval = ['attr3_1', 'sinc3_1', 'intel3_1', 'fun3_1', 'amb3_1']
eval_by_others = ["attr_o", "sinc_o", "intel_o", "fun_o", "amb_o"]

corr_eval = filterd_df[evaluation].corr('pearson')[auto_eval]
corr_df_fig = px.imshow(corr_eval, text_auto=True,
    color_continuous_scale="RdBu_r",
    title="Matrice de corrélation (Pearson)",
    zmin=-1, zmax=1)
corr_df_fig.show()

Il n'y a pas de correlation entre la valeur autoévaluée des participants et le valeur perçues par le partenaire lors du RDV.

### Non les personnes n'apprecient pas correctement leur propre valeur

- - - - - 

### Est ce que l'ordre des RDV est important pour avoir un second RDV ?

In [ ]:
taux_oui_par_ordre = filterd_df.groupby('order')[['dec','dec_o','match']].mean()
print(taux_oui_par_ordre)
taux_oui_par_ordre.reset_index(inplace=True)
fig = px.line(
    taux_oui_par_ordre, x="order", y=['dec','dec_o','match'],
    markers=True, title="Moyenne de 'Yes' et 'Match' selon l'ordre"
)

for col in ['dec', 'dec_o', 'match']:
    x = taux_oui_par_ordre['order']
    y = taux_oui_par_ordre[col]
    z = numpy.polyfit(x, y, 1)     
    p = numpy.poly1d(z)              
    fig.add_trace(go.Scatter(
        x=x, y=p(x),
        mode="lines",
        name=f"Tendance {col}",
        line={"dash":"solid"}
    ))

fig.show()


            dec     dec_o     match
order                              
1      0.496659  0.485523  0.229399
2      0.391982  0.396437  0.155902
3      0.423163  0.407572  0.171492
4      0.440980  0.438753  0.180401
5      0.414254  0.438753  0.167038
6      0.445434  0.434298  0.173719
7      0.421801  0.409953  0.172986
8      0.415865  0.420673  0.165865
9      0.443902  0.456098  0.202439
10     0.439791  0.426702  0.183246
11     0.416667  0.380000  0.156667
12     0.376667  0.426667  0.133333
13     0.386667  0.406667  0.143333
14     0.416667  0.443333  0.153333
15     0.377863  0.389313  0.122137
16     0.357759  0.336207  0.099138
17     0.361502  0.352113  0.098592
18     0.403756  0.380282  0.169014
19     0.442029  0.405797  0.152174
20     0.365385  0.375000  0.192308
21     0.337209  0.337209  0.069767
22     0.386364  0.386364  0.227273


Il semblerait que plus les participants avance dans les rendez-vous moins ils acceptent de second rdv.